# CPU Scheduling Algorithms – FCFS & SJK
**نویسنده: حسام صارمی‌زاده (Hesam Saremizade)**

**استاد: میلاد یثربی**

در این پروژه، دو الگوریتم زمان‌بندی کلاسیک پیاده‌سازی شده‌اند:

- FCFS: First Come First Serve
- SJN: Shortest Job Next

هر دو الگوریتم از context switch پشتیبانی می‌کنند و قابلیت نمایش جدول نهایی و نمودار گانت را دارند.


In [89]:
import pandas as pd
import matplotlib.pyplot as plt

## تابع دریافت فرایندها

تابعی برای دریافت اطلاعات فرایندها از کاربر و تبدیل آن‌ها به دیتافریم:




In [90]:
def get_processes():
    processes = []
    help = "yes"
    while help == "yes":
        Process_ID = input("Enter Process ID: ")
        Arrival_Time = int(input("Enter Arrival Time: "))
        Execution_Time = int(input("Enter Execution Time: "))
        process = {"Process_ID": Process_ID, "Arrival_Time": Arrival_Time, "Execution_Time": Execution_Time}
        processes.append(process)
        help = input("Do you want to continue? (Yes/No):").lower()
    df_processes = pd.DataFrame(processes)
    return df_processes



### بخش 3:  FCFS



In [91]:
class FCFS:
    def __init__(self, processes, context_switch=0):
        self.processes = processes
        self.context_switch = context_switch

    def fit(self):
        self.df_processes = self.processes.sort_values("Arrival_Time", ascending=True).reset_index(drop=True)
        service_time = []
        waiting_time = []
        for i in range(self.df_processes.shape[0]):
            if i == 0:
                service_time.append(self.df_processes["Arrival_Time"][i])
                waiting_time.append(0)
            else:
                service = self.df_processes["Execution_Time"][i - 1] + service_time[i - 1] + self.context_switch
                service_time.append(service)

                waiting = service_time[i] - self.df_processes["Arrival_Time"][i]
                waiting_time.append(waiting)

        self.df_processes["Service_Time"] = service_time
        self.df_processes["Waiting_Time"] = waiting_time
        self.df_processes["Completion_Time"] = self.df_processes["Service_Time"] + self.df_processes[
            "Execution_Time"]
        if self.context_switch != 0:
            cs_records = []
            for i in range(len(self.df_processes) - 1):
                cs_start = self.df_processes["Completion_Time"][i]
                cs_records.append({
                    "Process_ID": f"CS_{i + 1}",
                    "Arrival_Time": None,
                    "Execution_Time": self.context_switch,
                    "Service_Time": cs_start,
                    "Waiting_Time": None,
                    "Completion_Time": cs_start + self.context_switch,
                    "Is_Context_Switch": True
                })

            df_cs = pd.DataFrame(cs_records)
            self.df_processes["Is_Context_Switch"] = False
            self.df_processes = pd.concat([self.df_processes, df_cs], ignore_index=True)
            self.df_processes = self.df_processes.sort_values("Service_Time").reset_index(drop=True)

    def show_orginal(self):
        print("Original Input Data:")
        print(self.processes)

    def show_sort(self):
        print("Sorted Processes (excluding context switches):")
        if self.context_switch != 0:
            print(self.df_processes[self.df_processes.Is_Context_Switch == False])
        else:
            print(self.df_processes)

    def show_All(self):
        print("Full Process Table (including context switches):")
        print(self.df_processes)

    def show_Service_Time(self):
        print("Service Times:")
        if self.context_switch != 0:
            print(self.df_processes[self.df_processes.Is_Context_Switch == False][["Process_ID", "Service_Time"]])
        else:
            print(self.df_processes[["Process_ID", "Service_Time"]])

    def show_Waiting_Time(self):
        print("Waiting Times:")
        if self.context_switch != 0:
            print(self.df_processes[self.df_processes.Is_Context_Switch == False][["Process_ID", "Waiting_Time"]])
        else:
            print(self.df_processes[["Process_ID", "Waiting_Time"]])

    def show_Average_waiting_time(self):
        if "Is_Context_Switch" in self.df_processes.columns:
            avg = self.df_processes[self.df_processes.Is_Context_Switch == False]["Waiting_Time"].mean()
        else:
            avg = self.df_processes["Waiting_Time"].mean()
        print(f"Average Waiting Time: {avg}")

    def gantt(self):
        fig, ax = plt.subplots(figsize=(12, 3))
        y = 0

        for _, row in self.df_processes.iterrows():
            color = 'green' if row.get("Is_Context_Switch", False) else 'white'
            start = row["Service_Time"]
            duration = row["Execution_Time"]

            ax.barh(y, duration, left=start, height=0.5, color=color, edgecolor='black')
            ax.text(start + duration / 2, y, row["Process_ID"], ha='center', va='center', color='black', fontsize=9)

        ax.set_yticks([])
        ax.set_xlabel("Time")
        ax.set_title("Gantt Chart - FCFS")

        max_time = int(self.df_processes["Completion_Time"].max())
        ax.set_xlim(0, max_time)
        ax.set_xticks(range(0, max_time + 1))

        plt.tight_layout()
        plt.show()




###  بخش 4: SJK



In [92]:
class SJN:
    def __init__(self, processes, context_switch=0):
        self.processes = processes.copy()
        self.context_switch = context_switch

    def fit(self):
        df = self.processes.sort_values(["Arrival_Time", "Execution_Time"]).reset_index(drop=True)
        df["Is_Completed"] = False
        df["Service_Time"] = None
        df["Waiting_Time"] = None
        df["Completion_Time"] = None

        time = 0
        cs_records = []

        while not df["Is_Completed"].all():
            available = df[(df["Arrival_Time"] <= time) & (df["Is_Completed"] == False)]
            if available.empty:
                time += 1
                continue

            idx = available["Execution_Time"].idxmin()
            row = df.loc[idx]

            start = time
            finish = start + row["Execution_Time"]

            df.at[idx, "Service_Time"] = start
            df.at[idx, "Waiting_Time"] = start - row["Arrival_Time"]
            df.at[idx, "Completion_Time"] = finish
            df.at[idx, "Is_Completed"] = True

            time = finish
            if self.context_switch != 0:
                cs_records.append({
                    "Process_ID": f"CS_{len(cs_records) + 1}",
                    "Arrival_Time": None,
                    "Execution_Time": self.context_switch,
                    "Service_Time": time,
                    "Waiting_Time": None,
                    "Completion_Time": time + self.context_switch,
                    "Is_Context_Switch": True
                })
                time += self.context_switch

        df["Is_Context_Switch"] = False
        self.df_processes = pd.concat([df, pd.DataFrame(cs_records)], ignore_index=True)
        self.df_processes = self.df_processes.sort_values("Service_Time").reset_index(drop=True)


    def show_orginal(self):
        print("Original Input Data:")
        print(self.processes)

    def show_sort(self):
        print("Sorted Processes (excluding context switches):")
        if self.context_switch != 0:
            print(self.df_processes[self.df_processes.Is_Context_Switch == False])
        else:
            print(self.df_processes)

    def show_All(self):
        print("Full Process Table (including context switches):")
        print(self.df_processes)

    def show_Service_Time(self):
        print("Service Times:")
        if self.context_switch != 0:
            print(self.df_processes[self.df_processes.Is_Context_Switch == False][["Process_ID", "Service_Time"]])
        else:
            print(self.df_processes[["Process_ID", "Service_Time"]])

    def show_Waiting_Time(self):
        print("Waiting Times:")
        if self.context_switch != 0:
            print(self.df_processes[self.df_processes.Is_Context_Switch == False][["Process_ID", "Waiting_Time"]])
        else:
            print(self.df_processes[["Process_ID", "Waiting_Time"]])

    def show_Average_waiting_time(self):
        if "Is_Context_Switch" in self.df_processes.columns:
            avg = self.df_processes[self.df_processes.Is_Context_Switch == False]["Waiting_Time"].mean()
        else:
            avg = self.df_processes["Waiting_Time"].mean()
        print(f"Average Waiting Time: {avg}")

    def gantt(self):
        fig, ax = plt.subplots(figsize=(12, 3))
        y = 0

        for _, row in self.df_processes.iterrows():
            color = 'green' if row.get("Is_Context_Switch", False) else 'white'
            start = row["Service_Time"]
            duration = row["Execution_Time"]

            ax.barh(y, duration, left=start, height=0.5, color=color, edgecolor='black')
            ax.text(start + duration / 2, y, row["Process_ID"], ha='center', va='center', color='black', fontsize=9)

        ax.set_yticks([])
        ax.set_xlabel("Time")
        ax.set_title("Gantt Chart - SKJ")

        max_time = int(self.df_processes["Completion_Time"].max())
        ax.set_xlim(0, max_time)
        ax.set_xticks(range(0, max_time + 1))

        plt.tight_layout()
        plt.show()